<a href="https://colab.research.google.com/github/deregenboogm/NLP_coronavirus_project/blob/master/data_frames/s4_WHO_director_speeches.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def get_speeches():
  ## Define url
  url = "https://www.who.int/dg/speeches/detail"

  ## Request to access the url
  r = requests.get(url)
  
  ## Parsing the content
  soup = BeautifulSoup(r.content, 'html.parser')

  ## Find speeches and their links those are related to COVID-19
  items = soup.find_all("div", class_ = "list-view--item vertical-list-item")
  
  links = []
  for item in items:
    if "COVID-19" in item.p.text:
      links.append(item.a["href"])
  
  ## Optain a list of speeches   
  speeches=[]
  
  for link in links: 
    r = requests.get(link)
    soup = BeautifulSoup(r.content, "html.parser")

    speech = {}
    speech["title"] = soup.h1.text
    speech["date"] = soup.find("div", class_= "date").text
    speech["date"] = pd.to_datetime(speech["date"])
    speech["date"] = speech["date"].isoformat().split('T')[0]
    speech["link"] =  link
    speech["summary"] = []

    raw_text = []
    paragraphs = soup.find_all("p")
    for p in paragraphs:
      joined_p = "".join(p.text)
      joined_p = joined_p.replace("===", "")
      joined_p = joined_p.replace(" '", "'")
      joined_p = joined_p.replace(".", ". ")
      raw_text.append(joined_p)

    raw_text = ''.join(map(str, raw_text))

    summ = generate_summary(raw_text)
    speech["summary"] = summ
  
    speeches.append(speech)
          
  df = pd.DataFrame(speeches, index=None)
  
  # Saving the dataframe
  df.to_csv("/content/drive/My Drive/WHO_speeches.csv", index = False)

  print("Total number of COVID-19 speeches: " , len(speeches))
  print("The 'WHO_speeches.csv' files have been successfully updated for today!")
